In [2]:
import pandas as pd

In [ ]:
# Data to Load: Everything in {data} and {memedata}

In [3]:
df = pd.read_csv('data/dogeusdt.csv')
df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,LogReturns
0,1609459200000,0.004672,0.004701,0.004601,0.004679,27682073.0,1609462799999,1.289648e+05,852,11568593.0,5.390462e+04,0.000000
1,1609462800000,0.004678,0.004764,0.004676,0.004737,45350221.0,1609466399999,2.147129e+05,1199,23071849.0,1.092274e+05,0.012257
2,1609466400000,0.004731,0.004759,0.004719,0.004755,17278135.0,1609469999999,8.185994e+04,431,8858901.0,4.197215e+04,0.003919
3,1609470000000,0.004752,0.004999,0.004751,0.004978,165646699.0,1609473599999,8.097999e+05,4102,87779596.0,4.285427e+05,0.045831
4,1609473600000,0.004978,0.005000,0.004800,0.004894,147434756.0,1609477199999,7.231637e+05,3020,62547348.0,3.065995e+05,-0.017079
...,...,...,...,...,...,...,...,...,...,...,...,...
33581,1730401200000,0.163460,0.164640,0.158900,0.159330,94394789.0,1730404799999,1.525987e+07,44251,34020548.0,5.507691e+06,-0.025652
33582,1730404800000,0.159320,0.160920,0.156460,0.157670,116328972.0,1730408399999,1.848090e+07,47910,49803577.0,7.917604e+06,-0.010473
33583,1730408400000,0.157670,0.160920,0.156460,0.160400,95516633.0,1730411999999,1.520021e+07,39723,55684902.0,8.864325e+06,0.017166
33584,1730412000000,0.160400,0.162850,0.159970,0.161980,112635122.0,1730415599999,1.819356e+07,36094,51663382.0,8.346327e+06,0.009802
